In [10]:
from os import listdir
from os.path import isfile, join
import string
import nltk
import os
import re
import string
import pandas as pd
import numpy as np
from pathlib import PurePath,PureWindowsPath
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from itertools import chain
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
import pickle
warnings.filterwarnings("ignore")

In [11]:
def remove_metadata(lines):
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines


# In[3]:


def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)


# In[4]:


def lemmaLower(words):
    #lemmatization
    words = words.split()
    #we also normalize the cases of our words
    words = [word.lower() for word in words]
    #pos
    #tags = pos_tag(words)
    #tags = list(tags)
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    #removing stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
  
    joined_string = ' '.join(words)
    joined_string = re.sub (r'\b\w\b','', joined_string)
    
    return joined_string


# In[5]:


def prepare(path):
    f = open(path, 'r')
    text_lines = f.readlines()
    
    #removing the meta-data at the top of each document
    text_lines = remove_metadata(text_lines)
    
    #initiazing an array to hold all the words in a document
    doc_words = []
   
    #traverse over all the lines and tokenize each one with the help of helper function: tokenize_sentence
    
    doc_words.append(text_lines)
    

    return doc_words


# In[6]:


def Text_Cleaning(data):
    
    data= data.replace('\\n','')
    data= data.replace('\\t','')
    
    #replace any non-charachter by space
    data  = re.sub('[^a-zA-Z]', ' ', data)
  
    #remove html tages
    data = remove_html(data)
    #remove punctuation
    punctuationfree = "".join([i for i in data if i not in string.punctuation])
    data = punctuationfree
    data = [re.sub (r'\b\w\b','', data)]
    return data

In [12]:
cwd = os.getcwd()
my_path = r"C:\Users\future\Desktop\20-news\20_newsgroups"
os.chdir(my_path)
cwd = os.getcwd()
#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]
folders

['20_newsgroups',
 'alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [13]:
files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])
files

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\future\\Desktop\\20-news\\20_newsgroups\\20_newsgroups'

In [ ]:
pathname_list = []
for i in range(len(folders)):
    for f in files[i]:
        pathname_list.append(join(my_path, join(folders[i], f)))

pathname_list

In [ ]:

len(pathname_list)

In [ ]:
Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)
        
Y

In [ ]:
len(Y)

In [ ]:
# Creating X
list_of_words = []

for document in pathname_list:
        list_of_words.append(prepare(str(document)))

In [ ]:
# covert it into dataframe and create Y
df = pd.DataFrame(list_of_words, columns=['documents'])
df['category'] = Y

In [ ]:
#Text Cleaning
df["documents"] = df["documents"].apply(lambda x: Text_Cleaning(str(x)))
#remove extra spaces in text
df["documents"] = df["documents"].apply(lambda x: [" ".join(text.split()) for text in x])
df["documents"] = df["documents"].apply(lambda x: " ".join(x))
df.head()

In [ ]:
#spiliting data into train & test
#return the train test split to 0.20
X_train, X_test, y_train, y_test = train_test_split(df['documents'][:5000], df['category'][:5000], random_state=0, test_size=0.20)
len(X_train)

In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english',preprocessor = lemmaLower , max_features=55900 , min_df = 0.0001, max_df = 0.97)
tfidfvectorizer.fit(X_train)
X_train_tfidf = tfidfvectorizer.transform(X_train).toarray()
X_test_tfidf = tfidfvectorizer.transform(X_test).toarray()

In [ ]:
gnb = GaussianNB()  
y_pred = gnb.fit(X_train_tfidf, y_train).predict(X_test_tfidf)

print("Naive Bayes accuracy: ", gnb.score( X_test_tfidf, y_test)*100)


In [ ]:
clff = SVC(kernel='linear')
clff.fit(X_train_tfidf, y_train)
accuracy = clff.score(X_test_tfidf, y_test)

print(f"SVM Accuracy: {accuracy*100}")
 #Accuracy linear: 85.575
 #Accuracy poly: 78.64999999999999
 #Accuracy rbf: 85.35000000000001

In [ ]:
#KNN model

knn = KNeighborsClassifier(n_neighbors=1500)  # Specify the number of neighbors (K)
knn.fit(X_train_tfidf, y_train)
accuracy_knn = knn.score(X_test_tfidf, y_test)

print(f"KNN Accuracy: {accuracy_knn*100}")
#KNN Accuracy: 69.25

In [ ]:
# GradientBoostingClassifier model

#gbm = GradientBoostingClassifier()
#gbm.fit(X_train_tfidf, y_train)
#accuracy_gbm = gbm.score(X_test_tfidf, y_test)

#print(f"GBM Accuracy: {accuracy_gbm*100}")
#GBM Accuracy: 75.55

In [ ]:
# DecisionTree model

dtc = DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=3 , min_samples_leaf=5)
dtc = dtc.fit(X_train_tfidf,y_train)
y_pred = dtc.predict(X_test_tfidf)

print("Decision tree Accuracy:",accuracy_score(y_test, y_pred)*100)


# Predicting the values of test data
#print("Classification report - \n", classification_report(y_test,y_pred))

In [ ]:
#RandomForest model

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100}")
#Accuracy: 78.27499999999999

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(random_state=16)
log.fit(X_train_tfidf,y_train)
x_test_pred=log.predict(X_test_tfidf)
test_accuracy=accuracy_score(x_test_pred,y_test)

print("accuracy of testing data by Logistic Regression is : ",test_accuracy*100)


# In[ ]:

In [ ]:
import joblib
joblib_file="20_newsgroups"
joblib.dump(log,joblib_file)

loaded_model=joblib.load(open(joblib_file,'rb'))
x_test_pred=loaded_model.predict(X_test_tfidf)
result=np.round(accuracy_score(y_test,x_test_pred),2)
print(result)

In [ ]:
import joblib
joblib_file="20_newsgroups"
joblib.dump(clf,joblib_file)

loaded_model=joblib.load(open(joblib_file,'rb'))
x_test_pred=loaded_model.predict(X_test_tfidf)
result=np.round(accuracy_score(y_test,x_test_pred),2)
print(result)

In [ ]:
import joblib
joblib_file="20_newsgroups"
joblib.dump(gnb,joblib_file)

loaded_model=joblib.load(open(joblib_file,'rb'))
x_test_pred=loaded_model.predict(X_test_tfidf)
result=np.round(accuracy_score(y_test,x_test_pred),2)
print(result)

In [ ]:
import joblib
joblib_file="20_newsgroups"
joblib.dump(knn,joblib_file)

loaded_model=joblib.load(open(joblib_file,'rb'))
x_test_pred=loaded_model.predict(X_test_tfidf)
result=np.round(accuracy_score(y_test,x_test_pred),2)
print(result)

In [ ]:
import joblib
joblib_file="20_newsgroups"
joblib.dump(clff,joblib_file)

loaded_model=joblib.load(open(joblib_file,'rb'))
x_test_pred=loaded_model.predict(X_test_tfidf)
result=np.round(accuracy_score(y_test,x_test_pred),2)
print(result)

In [ ]:
#import joblib
#joblib_file="20_newsgroups"
#joblib.dump(gbm,joblib_file)

#loaded_model=joblib.load(open(joblib_file,'rb'))
#x_test_pred=loaded_model.predict(X_test_tfidf)
#result=np.round(accuracy_score(y_test,x_test_pred),2)
#print(result)